# COMP90089 Assignment 1: Digital Phenotypes

# Code

In [ ]:
# Enviroment Setup
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

# authenticate
auth.authenticate_user()

# Set up environment variables
project_id = 'model-wave-394405'
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
dataset = 'mimiciv'

# SQL code for data retrieving


In [ ]:
from google.cloud import bigquery

client = bigquery.Client()


In [ ]:
stroke_diagnoses_query = """
SELECT DISTINCT hadm_id
FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
WHERE icd_version = 10
AND (icd_code LIKE 'I63%' OR icd_code LIKE 'I61%')
"""

stroke_diagnoses = client.query(stroke_diagnoses_query).to_dataframe()
stroke_hadm_ids = stroke_diagnoses['hadm_id'].tolist()



In [ ]:
demographics_query = """
SELECT
    p.subject_id,
    p.gender,
    p.anchor_age AS age,
    a.race,
    a.hadm_id
FROM
    `physionet-data.mimiciv_hosp.patients` AS p
JOIN
    `physionet-data.mimiciv_hosp.admissions` AS a
ON
    p.subject_id = a.subject_id
WHERE
    a.hadm_id IN UNNEST(@hadm_ids)
"""

job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("hadm_ids", "INT64", stroke_hadm_ids)
    ]
)

demographics = client.query(demographics_query, job_config=job_config).to_dataframe()
print(demographics.shape)


(3966, 5)


In [ ]:
icu_stays_query = """
SELECT
    icu.subject_id,
    icu.hadm_id,
    icu.stay_id,
    icu.intime AS icu_intime,
    icu.outtime AS icu_outtime
FROM
    `physionet-data.mimiciv_icu.icustays` AS icu
WHERE
    icu.hadm_id IN UNNEST(@hadm_ids)
"""

icu_stays = client.query(icu_stays_query, job_config=job_config).to_dataframe()


In [ ]:
lab_items_query = """
SELECT itemid, label
FROM `physionet-data.mimiciv_hosp.d_labitems`
WHERE label IN ('Anion Gap', 'Bicarbonate', 'Chloride', 'Red Blood Cells', 'White Blood Cells')
"""

lab_items = client.query(lab_items_query).to_dataframe()
lab_itemids = lab_items['itemid'].tolist()
print(lab_items)


   itemid              label
0   50868          Anion Gap
1   50882        Bicarbonate
2   50902           Chloride
3   51755  White Blood Cells
4   51756  White Blood Cells
5   52500          Anion Gap
6   52535           Chloride
7   51279    Red Blood Cells
8   51301  White Blood Cells


In [ ]:
lab_events_query = """
SELECT
    le.subject_id,
    le.hadm_id,
    le.itemid,
    le.charttime,
    le.value,
    le.valuenum,
    le.valueuom
FROM
    `physionet-data.mimiciv_hosp.labevents` AS le
JOIN
    `physionet-data.mimiciv_icu.icustays` AS icu
ON
    le.subject_id = icu.subject_id
    AND le.hadm_id = icu.hadm_id
WHERE
    le.hadm_id IN UNNEST(@hadm_ids)
    AND le.itemid IN UNNEST(@lab_itemids)
    AND DATETIME_DIFF(le.charttime, icu.intime, HOUR) BETWEEN 0 AND 24
"""

lab_events_job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("hadm_ids", "INT64", stroke_hadm_ids),
        bigquery.ArrayQueryParameter("lab_itemids", "INT64", lab_itemids)
    ]
)

lab_events = client.query(lab_events_query, job_config=lab_events_job_config).to_dataframe()


In [ ]:
vital_items_query = """
SELECT itemid, label
FROM `physionet-data.mimiciv_icu.d_items`
WHERE label IN (
    'Heart Rate',
    'Respiratory Rate',
    'Non Invasive Blood Pressure systolic',
    'Non Invasive Blood Pressure diastolic',
    'SpO2'
)
"""

vital_items = client.query(vital_items_query).to_dataframe()
vital_itemids = vital_items['itemid'].tolist()


In [ ]:
print(vital_items)

   itemid                                  label
0  220210                       Respiratory Rate
1  220045                             Heart Rate
2  220179   Non Invasive Blood Pressure systolic
3  220180  Non Invasive Blood Pressure diastolic


In [ ]:
vital_signs_query = """
SELECT
    ce.subject_id,
    ce.hadm_id,
    ce.stay_id,
    ce.itemid,
    ce.charttime,
    ce.valuenum,
    ce.valueuom
FROM
    `physionet-data.mimiciv_icu.chartevents` AS ce
JOIN
    `physionet-data.mimiciv_icu.icustays` AS icu
ON
    ce.stay_id = icu.stay_id
WHERE
    ce.hadm_id IN UNNEST(@hadm_ids)
    AND ce.itemid IN UNNEST(@vital_itemids)
    AND DATETIME_DIFF(ce.charttime, icu.intime, HOUR) BETWEEN 0 AND 24
"""

vital_signs_job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("hadm_ids", "INT64", stroke_hadm_ids),
        bigquery.ArrayQueryParameter("vital_itemids", "INT64", vital_itemids)
    ]
)

vital_signs = client.query(vital_signs_query, job_config=vital_signs_job_config).to_dataframe()


In [ ]:
gcs_items_query = """
SELECT itemid, label
FROM `physionet-data.mimiciv_icu.d_items`
WHERE label IN (
    'GCS - Eye Opening',
    'GCS - Motor Response',
    'GCS - Verbal Response'
)
"""

gcs_items = client.query(gcs_items_query).to_dataframe()
gcs_itemids = gcs_items['itemid'].tolist()


In [ ]:
gcs_data_query = """
SELECT
    ce.subject_id,
    ce.hadm_id,
    ce.stay_id,
    ce.itemid,
    ce.charttime,
    ce.value,
    ce.valuenum
FROM
    `physionet-data.mimiciv_icu.chartevents` AS ce
JOIN
    `physionet-data.mimiciv_icu.icustays` AS icu
ON
    ce.stay_id = icu.stay_id
WHERE
    ce.hadm_id IN UNNEST(@hadm_ids)
    AND ce.itemid IN UNNEST(@gcs_itemids)
    AND DATETIME_DIFF(ce.charttime, icu.intime, HOUR) BETWEEN 0 AND 24
"""

gcs_data_job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("hadm_ids", "INT64", stroke_hadm_ids),
        bigquery.ArrayQueryParameter("gcs_itemids", "INT64", gcs_itemids)
    ]
)

gcs_data = client.query(gcs_data_query, job_config=gcs_data_job_config).to_dataframe()


In [ ]:
print(demographics.head())

print(icu_stays.head())

print(lab_events.head())

print(vital_signs.head())

print(gcs_data.head())


人口统计学数据:
   subject_id gender  age                         race   hadm_id
0    10822967      M   20       BLACK/AFRICAN AMERICAN  26681360
1    16006168      M   20  HISPANIC/LATINO - DOMINICAN  29449066
2    13419296      F   27                        WHITE  27921658
3    17962479      F   27                        WHITE  25206309
4    12342976      M   50                        WHITE  20894865

ICU 入住时间:
   subject_id   hadm_id   stay_id          icu_intime         icu_outtime
0    10004113  29879900  35200789 2173-03-20 20:16:36 2173-03-21 21:43:59
1    10103795  22741814  31411464 2176-07-01 19:45:36 2176-07-11 22:07:48
2    10107132  27344948  30455932 2176-03-26 08:40:00 2176-03-26 23:01:42
3    10132136  29646136  35896096 2160-01-31 05:49:00 2160-02-01 16:16:34
4    10253146  26685114  33113606 2183-05-09 23:41:00 2183-05-14 17:29:34

实验室检查结果:
   subject_id   hadm_id  itemid           charttime value  valuenum valueuom
0    10253146  26685114   51279 2183-05-10 09:15:00  3.80  

# Python code for data processing

In [ ]:
lab_events_agg = lab_events.groupby(['hadm_id', 'itemid']).agg(
    mean_value=('valuenum', 'mean'),
    std_value=('valuenum', 'std')
).reset_index()


In [ ]:
vital_signs_agg = vital_signs.groupby(['hadm_id', 'itemid']).agg(
    mean_value=('valuenum', 'mean'),
    std_value=('valuenum', 'std')
).reset_index()


In [ ]:
gcs_data_agg = gcs_data.groupby(['hadm_id', 'itemid']).agg(
    mean_value=('valuenum', 'mean'),
    std_value=('valuenum', 'std')
).reset_index()


In [ ]:
lab_pivot = lab_events_agg.pivot_table(
    index='hadm_id',
    columns='itemid',
    values=['mean_value', 'std_value']
)

lab_pivot.columns = [f'lab_{func}_{itemid}' for func, itemid in lab_pivot.columns]
lab_pivot = lab_pivot.reset_index()


In [ ]:
vital_pivot = vital_signs_agg.pivot_table(
    index='hadm_id',
    columns='itemid',
    values=['mean_value', 'std_value']
)

vital_pivot.columns = [f'vital_{func}_{itemid}' for func, itemid in vital_pivot.columns]
vital_pivot = vital_pivot.reset_index()


In [ ]:
gcs_pivot = gcs_data_agg.pivot_table(
    index='hadm_id',
    columns='itemid',
    values=['mean_value', 'std_value']
)

gcs_pivot.columns = [f'gcs_{func}_{itemid}' for func, itemid in gcs_pivot.columns]
gcs_pivot = gcs_pivot.reset_index()


In [ ]:
mortality_info_query = """
SELECT
    hadm_id,
    CASE WHEN deathtime IS NOT NULL THEN 1 ELSE 0 END AS mortality
FROM
    `physionet-data.mimiciv_hosp.admissions`
WHERE
    hadm_id IN UNNEST(@hadm_ids)
"""

job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ArrayQueryParameter("hadm_ids", "INT64", stroke_hadm_ids)
    ]
)

mortality_info = client.query(mortality_info_query, job_config=job_config).to_dataframe()


In [ ]:
print(mortality_info)
mortality_rate = mortality_info['mortality'].mean()
print(f"Mortality Rate: {mortality_rate}")
print(icu_stays.head())

       hadm_id  mortality
0     28237094          1
1     28899895          0
2     27627470          0
3     28844727          0
4     29299221          0
...        ...        ...
3961  23920941          1
3962  29266081          0
3963  27334428          0
3964  27282483          0
3965  25307835          0

[3966 rows x 2 columns]
Mortality Rate: 0.12884518406454867
   subject_id   hadm_id   stay_id          icu_intime         icu_outtime
0    10004113  29879900  35200789 2173-03-20 20:16:36 2173-03-21 21:43:59
1    10103795  22741814  31411464 2176-07-01 19:45:36 2176-07-11 22:07:48
2    10107132  27344948  30455932 2176-03-26 08:40:00 2176-03-26 23:01:42
3    10132136  29646136  35896096 2160-01-31 05:49:00 2160-02-01 16:16:34
4    10253146  26685114  33113606 2183-05-09 23:41:00 2183-05-14 17:29:34


In [ ]:
print(f"demographics_icu hadm_id 数量：{demographics['hadm_id'].nunique()}")
print(f"demographics_icu 行数：{len(demographics)}")


demographics_icu hadm_id 数量：3966
demographics_icu 行数：3966


In [ ]:
data = demographics.merge(lab_pivot, on='hadm_id', how='left')
data = data.merge(vital_pivot, on='hadm_id', how='left')
data = data.merge(gcs_pivot, on='hadm_id', how='left')
data = data.merge(mortality_info, on='hadm_id', how='left')

In [ ]:
print(data.shape)
print(data.head())

(3966, 30)
   subject_id gender  age                         race   hadm_id  \
0    10822967      M   20       BLACK/AFRICAN AMERICAN  26681360   
1    16006168      M   20  HISPANIC/LATINO - DOMINICAN  29449066   
2    13419296      F   27                        WHITE  27921658   
3    17962479      F   27                        WHITE  25206309   
4    12342976      M   50                        WHITE  20894865   

   lab_mean_value_50868  lab_mean_value_50882  lab_mean_value_50902  \
0                   NaN                   NaN                   NaN   
1                   NaN                   NaN                   NaN   
2                  12.0                  19.5                 108.0   
3                  15.0                  22.0                 105.0   
4                  15.0                  23.5                 102.0   

   lab_mean_value_51279  lab_mean_value_51301  ...  vital_std_value_220179  \
0                   NaN                   NaN  ...                     NaN 

# Feature Selection

In [ ]:
missing_values = data.isna().mean()
print(missing_values)


subject_id                 0.000000
gender                     0.000000
age                        0.000000
race                       0.000000
hadm_id                    0.000000
lab_mean_value_50868       0.425618
lab_mean_value_50882       0.425618
lab_mean_value_50902       0.425618
lab_mean_value_51279       0.427887
lab_mean_value_51301       0.427887
lab_std_value_50868        0.694655
lab_std_value_50882        0.694150
lab_std_value_50902        0.690368
lab_std_value_51279        0.725920
lab_std_value_51301        0.725920
vital_mean_value_220045    0.389813
vital_mean_value_220179    0.415280
vital_mean_value_220180    0.415280
vital_mean_value_220210    0.391326
vital_std_value_220045     0.390318
vital_std_value_220179     0.424609
vital_std_value_220180     0.424609
vital_std_value_220210     0.392587
gcs_mean_value_220739      0.390822
gcs_mean_value_223900      0.390822
gcs_mean_value_223901      0.390822
gcs_std_value_220739       0.396621
gcs_std_value_223900       0

In [ ]:
continuous_features = [col for col in data.columns if data[col].dtype != 'object']
data[continuous_features] = data[continuous_features].fillna(data[continuous_features].mean())

categorical_features = [col for col in data.columns if data[col].dtype == 'object']
data[categorical_features] = data[categorical_features].fillna(data[categorical_features].mode().iloc[0])


In [ ]:
print(data[continuous_features].describe())

for col in categorical_features:
    print(data[col].value_counts())


            subject_id        age          hadm_id  lab_mean_value_50868  \
count           3966.0     3966.0           3966.0           3966.000000   
mean   15056943.998739  67.659859  24945685.469743             15.009737   
std     2897755.593076  15.020458   2855128.522669              2.450135   
min         10003299.0       18.0       20002506.0              6.000000   
25%         12542518.0       58.0      22486773.25             14.000000   
50%         15044222.0       69.0       24953339.5             15.009737   
75%         17605186.5       79.0       27331772.0             15.009737   
max         19989126.0       91.0       29999625.0             37.200000   

       lab_mean_value_50882  lab_mean_value_50902  lab_mean_value_51279  \
count           3966.000000           3966.000000           3966.000000   
mean              22.786547            103.681720              3.877743   
std                2.492992              3.826906              0.555621   
min            

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# for col in continuous_features:
#     sns.histplot(data[col], kde=True)
#     plt.title(f'Distribution of {col}')
#     plt.show()


In [ ]:
correlations = []

for col in continuous_features:
    valid_idx = data[col].notnull() & data['mortality'].notnull()
    x = data.loc[valid_idx, col]
    y = data.loc[valid_idx, 'mortality']

    corr, p_value = pointbiserialr(x, y)
    correlations.append({
        'variable': col,
        'correlation': corr,
        'p_value': p_value
    })

corr_df = pd.DataFrame(correlations)
corr_df['abs_correlation'] = corr_df['correlation'].abs()

corr_df = corr_df.sort_values(by='abs_correlation', ascending=False)

print(corr_df[['variable', 'correlation', 'p_value']])


                   variable  correlation        p_value
27                mortality     1.000000   0.000000e+00
23    gcs_mean_value_223901    -0.470802  5.415689e-218
21    gcs_mean_value_220739    -0.457162  4.282939e-204
22    gcs_mean_value_223900    -0.411594  5.155805e-162
17   vital_std_value_220045     0.195609   1.683956e-35
26     gcs_std_value_223901     0.179771   3.692452e-30
3      lab_mean_value_50868     0.131797   7.825170e-17
4      lab_mean_value_50882    -0.128017   5.832038e-16
7      lab_mean_value_51301     0.126874   1.057848e-15
13  vital_mean_value_220045     0.126333   1.399789e-15
18   vital_std_value_220179     0.123329   6.484735e-15
6      lab_mean_value_51279    -0.117182   1.332693e-13
16  vital_mean_value_220210     0.109749   4.198383e-12
14  vital_mean_value_220179    -0.106125   2.081706e-11
1                       age     0.102118   1.149805e-10
10      lab_std_value_50902     0.084890   8.601192e-08
8       lab_std_value_50868     0.077136   1.153